# Cleaning the data and providing a small and easily parsable database.

The database with the full logs is too huge to work on. Most of the data would only be needed for debugging but not for evaluation.
Thus, we create a clean and small database for the further evaluation.

In [1]:
from _conf import ITERATIONS, ITERATION_TIME_LIMIT, TIME_LIMIT, BASE, RESULT_FOLDER, INPUT_SAMPLE_ARCHIVE, INSTANCE_ARCHIVE

import pandas as pd
from algbench import read_as_pandas, Benchmark
from _utils import parse_solution_overview

In [2]:
# Read the baseline results for comparison. They also already contain instance data (num_clauses and num_features).
baseline_data = pd.read_json("./00_simple_baseline_data.json.zip")
baseline_data.rename(columns={"sample_size": "baseline_sample_size"}, inplace=True)
baseline_data

,instance_name,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime,baseline_sample_size,path_to_baseline_sample
0,calculate,9,15,FIDE-ICPL,t2,1.484,9.0,2023-03-01_13-51-03/1_1_0_1_sample.csv
1,calculate,9,15,FIDE-ICPL,t2,1.280,9.0,2023-03-01_13-51-03/1_1_0_2_sample.csv
2,calculate,9,15,FIDE-ICPL,t2,1.289,9.0,2023-03-01_13-51-03/1_1_0_3_sample.csv
3,calculate,9,15,FIDE-ICPL,t2,1.274,9.0,2023-03-01_13-51-03/1_1_0_4_sample.csv
4,calculate,9,15,FIDE-ICPL,t2,1.277,9.0,2023-03-01_13-51-03/1_1_0_5_sample.csv
...,...,...,...,...,...,...,...,...
2745,freetz,31012,102705,YASA,t2_m10_null,900.008,NaN,None
2746,freetz,31012,102705,YASA,t2_m10_null,900.008,NaN,None
2747,freetz,31012,102705,YASA,t2_m10_null,900.010,NaN,None
2748,freetz,31012,102705,YASA,t2_m10_null,900.009,NaN,None


In [3]:
# Get the data from the result folder
samplns_data = read_as_pandas(
        RESULT_FOLDER,
        lambda result: {
            "parameters": result["parameters"],
            "initial_sample_path": result["parameters"]["args"]["initial_sample_path"],
            "instance_name": result["parameters"]["args"]["instance_name"],
            "lower_bound": result["result"]["lower_bound"],
            "upper_bound": len(result["result"]["solution"]),
            "iteration_info": result["result"]["iteration_info"],
            "time_used_by_yasa": result["result"]["time_used_by_yasa"],
            "timelimit_for_samplns": result["result"]["timelimit_for_samplns"],
            "samplns_used": result["result"]["samplns_used"],
            "runtime": result["runtime"],
            "time_limit": result["parameters"]["args"]["time_limit"],
        } if result.get("result", None) and  result.get("result", dict()).get("solution", None) else None,
    )
samplns_data.drop_duplicates(subset=["initial_sample_path"], inplace=True)
samplns_data

,parameters,initial_sample_path,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime,time_limit
0,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/44_1_7_1_sample.csv,XSEngine,32.0,40,"[{'nbrhd_tuples': 210, 'nbrhd_confs': 9, 'iter...",3.395,3596.605,True,3708.056571,3600
1,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/4_1_7_2_sample.csv,ChatClient,7.0,7,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 8, 'iter...",0.183,3599.817,True,15.838238,3600
2,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/47_1_7_2_sample.csv,ea2468,24.0,40,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 5, 'iter...",3.594,3596.406,True,3614.455043,3600
3,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/15_1_7_2_sample.csv,dell,31.0,31,"[{'nbrhd_tuples': 249, 'nbrhd_confs': 25, 'ite...",0.257,3599.743,True,117.523291,3600
4,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/25_1_7_5_sample.csv,fiasco_2020-12-01_14-09-14,196.0,196,"[{'nbrhd_tuples': 235, 'nbrhd_confs': 15, 'ite...",1.176,3598.824,True,2319.812010,3600
...,...,...,...,...,...,...,...,...,...,...,...
225,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/12_1_7_2_sample.csv,APL-Model,8.0,8,"[{'nbrhd_tuples': 249, 'nbrhd_confs': 12, 'ite...",0.192,3599.808,True,93.377796,3600
226,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/26_1_7_1_sample.csv,uclibc_2008-06-05_13-46-47,505.0,505,"[{'nbrhd_tuples': 243, 'nbrhd_confs': 44, 'ite...",1.492,3598.508,True,726.094270,3600
227,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/24_1_7_4_sample.csv,fiasco_2017-09-26_11-30-56,224.0,225,"[{'nbrhd_tuples': 234, 'nbrhd_confs': 22, 'ite...",0.990,3599.010,True,3655.331330,3600
228,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/2_1_7_3_sample.csv,lcm,6.0,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 8, 'itera...",0.202,3599.798,True,0.076086,3600


In [4]:
# Pull the baseline info into the data frame
samplns_data_with_baseline_infos = baseline_data.merge(right=samplns_data, left_on="path_to_baseline_sample", right_on="initial_sample_path", how="right", suffixes=("_baseline", "_samplns"))
samplns_data_with_baseline_infos.drop(columns=["instance_name_baseline", "initial_sample_path"], inplace=True)
samplns_data_with_baseline_infos.rename(columns={"instance_name_samplns": "instance_name"}, inplace=True)
samplns_data_with_baseline_infos

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,parameters,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime_samplns,time_limit
0,1273,2942,YASA,t2_m1_null,3.395,69.0,2023-03-01_13-51-03/44_1_7_1_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",XSEngine,32.0,40,"[{'nbrhd_tuples': 210, 'nbrhd_confs': 9, 'iter...",3.395,3596.605,True,3708.056571,3600
1,14,20,YASA,t2_m1_null,0.183,8.0,2023-03-01_13-51-03/4_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",ChatClient,7.0,7,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 8, 'iter...",0.183,3599.817,True,15.838238,3600
2,1408,3319,YASA,t2_m1_null,3.594,72.0,2023-03-01_13-51-03/47_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,24.0,40,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 5, 'iter...",3.594,3596.406,True,3614.455043,3600
3,46,244,YASA,t2_m1_null,0.257,38.0,2023-03-01_13-51-03/15_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",dell,31.0,31,"[{'nbrhd_tuples': 249, 'nbrhd_confs': 25, 'ite...",0.257,3599.743,True,117.523291,3600
4,258,1542,YASA,t2_m1_null,1.176,224.0,2023-03-01_13-51-03/25_1_7_5_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2020-12-01_14-09-14,196.0,196,"[{'nbrhd_tuples': 235, 'nbrhd_confs': 15, 'ite...",1.176,3598.824,True,2319.812010,3600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,28,40,YASA,t2_m1_null,0.192,15.0,2023-03-01_13-51-03/12_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",APL-Model,8.0,8,"[{'nbrhd_tuples': 249, 'nbrhd_confs': 12, 'ite...",0.192,3599.808,True,93.377796,3600
226,263,1699,YASA,t2_m1_null,1.492,510.0,2023-03-01_13-51-03/26_1_7_1_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",uclibc_2008-06-05_13-46-47,505.0,505,"[{'nbrhd_tuples': 243, 'nbrhd_confs': 44, 'ite...",1.492,3598.508,True,726.094270,3600
227,230,1181,YASA,t2_m1_null,0.990,246.0,2023-03-01_13-51-03/24_1_7_4_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2017-09-26_11-30-56,224.0,225,"[{'nbrhd_tuples': 234, 'nbrhd_confs': 22, 'ite...",0.990,3599.010,True,3655.331330,3600
228,9,16,YASA,t2_m1_null,0.202,8.0,2023-03-01_13-51-03/2_1_7_3_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",lcm,6.0,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 8, 'itera...",0.202,3599.798,True,0.076086,3600


In [5]:
# Check that we have all results.
samplns_data_with_baseline_infos.groupby(['instance_name']).size()

instance_name
APL                            5
APL-Model                      5
BankingSoftware                5
BattleofTanks                  5
ChatClient                     5
DMIE                           5
E-Shop                         5
EMBToolkit                     5
FameDB                         5
FeatureIDE                     5
FreeBSD-8_0_0                  5
PPU                            5
SafeBali                       5
SortingLine                    5
TightVNC                       5
Violet                         5
WaterlooGenerated              5
XSEngine                       5
aaed2000                       5
am31_sim                       5
atlas_mips32_4kc               5
axTLS                          5
berkeleyDB1                    5
berkeleyDB2                    5
busybox-1_18_0                 5
busybox-1_29_2                 5
busybox_2007-01-24_09-14-09    5
busybox_2020-12-16_21-53-05    5
calculate                      5
car                          

In [6]:
# Will stop the notebook if the data is bad
assert samplns_data_with_baseline_infos.groupby(['instance_name']).size().min() == 5
assert samplns_data_with_baseline_infos.groupby(['instance_name']).size().max() == 5

In [7]:
# Save the cleaned data to a json file
samplns_data_with_baseline_infos.to_json("./05_clean_data.json.zip")

In [8]:
# This just for looking into one instance.
t = read_as_pandas(
        RESULT_FOLDER,
        lambda result: {
            "parameters": result["parameters"],
            "initial_sample_path": result["parameters"]["args"]["initial_sample_path"],
            "instance_name": result["parameters"]["args"]["instance_name"],
            "lower_bound": result["result"]["lower_bound"],
            "upper_bound": len(result["result"]["solution"]),
            "iteration_info": result["result"]["iteration_info"],
            "time_used_by_yasa": result["result"]["time_used_by_yasa"],
            "timelimit_for_samplns": result["result"]["timelimit_for_samplns"],
            "samplns_used": result["result"]["samplns_used"],
            "instance": result["parameters"]["args"]["instance_name"],
            "logging": result["logging"],
        } if result.get("result", None) and  result.get("result", dict()).get("solution", None) else None,
    )
t = t[t["instance_name"] == "toybox_2020-12-06_00-02-46"]
for e in t.iloc[0]["logging"]:
    print(e["message"])

Parsed instance 'models/toybox_2020-12-06_00-02-46' with 334 features and 92 rules.
Preprocessing instance (Instance[models/toybox_2020-12-06_00-02-46]<334 features, 92 rules>).
Converting instance to CNF (Instance[models/toybox_2020-12-06_00-02-46|EQ]<334 features, 92 rules>).
Finished converting instance to CNF (Instance[models/toybox_2020-12-06_00-02-46|EQ|CNF]<334 features, 92 rules>).
Finnished preprocessing (Instance[models/toybox_2020-12-06_00-02-46|EQ|CNF]<334 features, 92 rules>).
Setting up random neighborhood selector.
Computing feasible tuples...
Converted sample to list representation.
Instance has 206665 feasible tuples.
Neighborhood selector is ready.
Building transaction graph for models/toybox_2020-12-06_00-02-46|EQ|CNF with 334 concrete features!
All valid configurations were added to the transaction graph.
Setting up random neighborhood selector.
Computing feasible tuples...
Converted sample to list representation.
Instance has 206665 feasible tuples.
Neighborhood se

In [9]:
# Check that we can read the data
data = pd.read_json("./05_clean_data.json.zip")
data

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,parameters,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime_samplns,time_limit
0,1273,2942,YASA,t2_m1_null,3.395,69,2023-03-01_13-51-03/44_1_7_1_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",XSEngine,32,40,"[{'nbrhd_tuples': 210, 'nbrhd_confs': 9, 'iter...",3.395,3596.605,True,3708.056571,3600
1,14,20,YASA,t2_m1_null,0.183,8,2023-03-01_13-51-03/4_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",ChatClient,7,7,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 8, 'iter...",0.183,3599.817,True,15.838238,3600
2,1408,3319,YASA,t2_m1_null,3.594,72,2023-03-01_13-51-03/47_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,24,40,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 5, 'iter...",3.594,3596.406,True,3614.455043,3600
3,46,244,YASA,t2_m1_null,0.257,38,2023-03-01_13-51-03/15_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",dell,31,31,"[{'nbrhd_tuples': 249, 'nbrhd_confs': 25, 'ite...",0.257,3599.743,True,117.523291,3600
4,258,1542,YASA,t2_m1_null,1.176,224,2023-03-01_13-51-03/25_1_7_5_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2020-12-01_14-09-14,196,196,"[{'nbrhd_tuples': 235, 'nbrhd_confs': 15, 'ite...",1.176,3598.824,True,2319.812010,3600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,28,40,YASA,t2_m1_null,0.192,15,2023-03-01_13-51-03/12_1_7_2_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",APL-Model,8,8,"[{'nbrhd_tuples': 249, 'nbrhd_confs': 12, 'ite...",0.192,3599.808,True,93.377796,3600
226,263,1699,YASA,t2_m1_null,1.492,510,2023-03-01_13-51-03/26_1_7_1_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",uclibc_2008-06-05_13-46-47,505,505,"[{'nbrhd_tuples': 243, 'nbrhd_confs': 44, 'ite...",1.492,3598.508,True,726.094270,3600
227,230,1181,YASA,t2_m1_null,0.990,246,2023-03-01_13-51-03/24_1_7_4_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",fiasco_2017-09-26_11-30-56,224,225,"[{'nbrhd_tuples': 234, 'nbrhd_confs': 22, 'ite...",0.990,3599.010,True,3655.331330,3600
228,9,16,YASA,t2_m1_null,0.202,8,2023-03-01_13-51-03/2_1_7_3_sample.csv,"{'func': 'run_samplns', 'args': {'instance_nam...",lcm,6,6,"[{'nbrhd_tuples': 37, 'nbrhd_confs': 8, 'itera...",0.202,3599.798,True,0.076086,3600


* `instance_name`: The instance name.
* `num_vars`: The number of variables in the instance.
* `num_clauses`: The number of clauses in the instance.
* `baseline_alg`: The baseline algorithm that created the sample
* `baseline_alg_conf`: The configuration of the baseline algorithm that created the sample.
* `runtime_baseline`: The runtime of the baseline algorithm that created the sample.
* `baseline_sample_size`: The size of the sample created by the baseline algorithm.
* `path_to_baseline_sample`: The path to the sample created by the baseline algorithm.
* `parameters` The parameters used to run samplns.
* `lower_bound` The lower bound computed by SampLNS.
* `upper_bound` The upper bound computed by SampLNS.
* `iteration_info` Info the extract the progress of SampLNS. The time do not include the baseline algorithm.
* `time_used_by_yasa` The time used by the baseline algorithm.
* `timelimit_for_samplns` The time limit for SampLNS. Check also `parameters` for the full timelimit.
* `runtime_samplns` The runtime of SampLNS without the baseline.
* `time_limit` The overall timelimit.
